# 1. Quantile Regression PyTorch (tabular data only)
This notebook generates baseline predictions using tabular data only. I created it to better understand the data.

It is a simplified version of [this great notebook](Osic-Multiple-Quantile-Regression-Starter) from [Ulrich GOUE
](https://www.kaggle.com/ulrich07). It also builds on this great [tutorial about Quantile Regression for neural networks](https://medium.com/the-artificial-impostor/quantile-regression-part-2-6fdbc26b2629).

# 2. Imports and global variables

In [1]:
import copy
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import GroupKFold
from torch.utils.data import Dataset
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader, Subset
import torch.nn as nn
import torch.nn.functional as F
from tqdm.notebook import trange
from time import time

In [2]:
root_dir = Path('/kaggle/input/osic-pulmonary-fibrosis-progression')
model_dir = Path('/kaggle/working')
num_kfolds = 5
batch_size = 128
learning_rate = 1e-4
num_epochs = 100
quantiles = (0.2, 0.5, 0.8)
model_name ='bernoulli_v1'

# 3. Dataset interface

In [3]:
torch.cuda.is_available()


True

In [4]:
torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

device(type='cuda', index=0)

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
class ClinicalDataset(Dataset):
    def __init__(self, root_dir, mode, transform=None):
        self.transform = transform
        self.mode = mode
        
        # https://www.kaggle.com/ulrich07/osic-multiple-quantile-regression-starter
        tr = pd.read_csv(Path(root_dir)/"train.csv")
        tr.drop_duplicates(keep=False, inplace=True, subset=['Patient','Weeks'])
        chunk = pd.read_csv(Path(root_dir)/"test.csv")

        sub = pd.read_csv(Path(root_dir)/"sample_submission.csv")
        sub['Patient'] = sub['Patient_Week'].apply(lambda x:x.split('_')[0])
        sub['Weeks'] = sub['Patient_Week'].apply(lambda x: int(x.split('_')[-1]))
        sub = sub[['Patient', 'Weeks', 'Confidence', 'Patient_Week']]
        sub = sub.merge(chunk.drop('Weeks', axis=1), on="Patient")

        tr['WHERE'] = 'train'
        chunk['WHERE'] = 'val'
        sub['WHERE'] = 'test'
        data = tr.append([chunk, sub])

        data['min_week'] = data['Weeks']
        data.loc[data.WHERE == 'test', 'min_week'] = np.nan
        data['min_week'] = data.groupby('Patient')['min_week'].transform('min')

        base = data.loc[data.Weeks == data.min_week]
        base = base[['Patient','FVC']].copy()
        base.columns = ['Patient', 'min_FVC']
        base['nb'] = 1
        base['nb'] = base.groupby('Patient')['nb'].transform('cumsum')
        base = base[base.nb==1]
        base.drop('nb', axis=1, inplace=True)

        data = data.merge(base, on='Patient', how='left')
        data['base_week'] = data['Weeks'] - data['min_week']
        del base

        COLS = ['Sex', 'SmokingStatus']
        self.FE = []
        for col in COLS:
            for mod in data[col].unique():
                self.FE.append(mod)
                data[mod] = (data[col] == mod).astype(int)

        data['age'] = (data['Age'] - data['Age'].min()) / \
                      (data['Age'].max() - data['Age'].min())
        data['BASE'] = (data['min_FVC'] - data['min_FVC'].min()) / \
                       (data['min_FVC'].max() - data['min_FVC'].min())
        data['week'] = (data['base_week'] - data['base_week'].min()) / \
                       (data['base_week'].max() - data['base_week'].min())
        data['percent'] = (data['Percent'] - data['Percent'].min()) / \
                          (data['Percent'].max() - data['Percent'].min())
        self.FE += ['age', 'percent', 'week', 'BASE']

        self.raw = data.loc[data.WHERE == mode].reset_index()
        del data

    def __len__(self):
        return len(self.raw)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        sample = {'features': self.raw[self.FE].iloc[idx].values,
                  'target': self.raw['FVC'].iloc[idx]}

        if self.transform:
            sample = self.transform(sample)

        return sample

# 4. Neural Net model and Quantile loss

In [7]:
class QuantModel(nn.Module):
    def __init__(self, in_features=9, out_features=3):
        super(QuantModel, self).__init__()
        self.fc1 = nn.Linear(in_features, 1000)
        self.fc2 = nn.Linear(1000, 500)
        self.fc3 = nn.Linear(500, out_features)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def quantile_loss(preds, target, quantiles):
    assert not target.requires_grad
    assert preds.size(0) == target.size(0)
    losses = []
    for i, q in enumerate(quantiles):
        errors = target - preds[:, i]
        losses.append(torch.max((q - 1) * errors, q * errors).unsqueeze(1))
        
    loss = torch.mean(torch.sum(torch.cat(losses, dim=1), dim=1))
    return loss

# 5. Training

In [8]:
# Helper generator that yields kfold PyTorch datasets
def group_kfold(dataset, groups, n_splits):
    gkf = GroupKFold(n_splits=n_splits)
    for train_idx, val_idx in gkf.split(dataset.raw, dataset.raw, groups):
        train = Subset(dataset, train_idx)
        val = Subset(dataset, val_idx)
        yield train, val
        
# Helper function with competition metric
def metric(preds, targets):
    sigma = preds[:, 2] - preds[:, 0]
    sigma[sigma < 70] = 70
    delta = (preds[:, 1] - targets).abs()
    delta[delta > 1000] = 1000
    return -np.sqrt(2) * delta / sigma - torch.log(np.sqrt(2) * sigma)

In [9]:
models = []

# Load the data
data = ClinicalDataset(root_dir, mode='train')
folds = group_kfold(data, data.raw['Patient'], num_kfolds)
t0 = time()

for fold, (trainset, valset) in enumerate(folds):
    # Prepare to save model weights
    Path(model_dir).mkdir(parents=True, exist_ok=True)
    now = datetime.now()
    fname = f'{model_name}-{now.year}{now.month:02d}{now.day:02d}_{fold}.pth'
    model_file = Path(model_dir) / fname

    dataset_sizes = {'train': len(trainset), 'val': len(valset)}
    dataloaders = {
        'train': DataLoader(trainset, batch_size=batch_size,
                            shuffle=True, num_workers=2),
        'val': DataLoader(valset, batch_size=batch_size,
                          shuffle=False, num_workers=2)
    }

    # Create the model and optimizer
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = QuantModel().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Set global tracking variables
    epoch_loss = {'train': np.inf, 'val': np.inf}
    epoch_metric = {'train': -np.inf, 'val': -np.inf}
    best_loss = np.inf
    best_model_wts = None
    df = pd.DataFrame(columns=['epoch', 'train_loss', 'val_loss'])

    # Training loop
    bar = trange(num_epochs, desc=f'Training fold {fold + 1}')
    for epoch in bar:
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_metric = 0.0
            # Iterate over data
            for batch in dataloaders[phase]:
                inputs = batch['features'].float().to(device)
                targets = batch['target'].to(device)

                # zero the parameter gradients
                optimizer.zero_grad()
                # forward
                # track gradients if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    preds = model(inputs)
                    loss = quantile_loss(preds, targets, quantiles)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_metric += metric(preds, targets).sum()

            # epoch statistics
            epoch_loss[phase] = running_loss / dataset_sizes[phase]
            epoch_metric[phase] = running_metric / dataset_sizes[phase]
            bar.set_postfix(a_train_loss=f'{epoch_loss["train"]:0.1f}',
                            b_val_loss=f'{epoch_loss["val"]:0.1f}',
                            c_train_metric=f'{epoch_metric["train"]:0.4f}',
                            d_val_metric=f'{epoch_metric["val"]:0.4f}')

            # deep copy the model
            if phase == 'val' and epoch_loss['val'] < best_loss:
                best_loss = epoch_loss['val']
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(best_model_wts, model_file)

        df = df.append({
            'epoch': epoch + 1,
            'train_loss': epoch_loss["train"],
            'val_loss': epoch_loss["val"]
        }, ignore_index=True)

    # Save training statistics
    fname = f'{model_name}-{now.year}{now.month:02d}{now.day:02d}_{fold}.csv'
    csv_file = Path(model_dir) / fname
    df.to_csv(csv_file, index=False)

    # load best model weights
    model.load_state_dict(best_model_wts)
    models.append(model)

print(f'Training complete! Time: {timedelta(seconds=time() - t0)}')


Training complete! Time: 0:13:47.946939


# 6. Generating submission CSV

In [10]:
data = ClinicalDataset(root_dir, mode='test')
avg_preds = torch.zeros((len(data), len(quantiles)))

for model in models:
    dataloader = DataLoader(data, batch_size=batch_size,
                            shuffle=False, num_workers=2)
    preds = []
    for batch in dataloader:
        inputs = batch['features'].float().to(device)
        with torch.no_grad():
            x = model(inputs)
            preds.append(x)

    preds = torch.cat(preds, dim=0).float().to("cpu")
    avg_preds += preds

avg_preds /= len(models)
avg_preds = avg_preds.numpy()
df = pd.DataFrame(data=avg_preds, columns=list(quantiles))
df['Patient_Week'] = data.raw['Patient_Week']
df['FVC'] = df[quantiles[1]]
df['Confidence'] = df[quantiles[2]] - df[quantiles[0]]
df = df.drop(columns=list(quantiles))
df.to_csv('submission.csv', index=False)

In [11]:
print(len(df))
df.head()

730


,Patient_Week,FVC,Confidence
0,ID00419637202311204720264_-12,2730.651611,912.923584
1,ID00419637202311204720264_-11,2734.111816,914.081299
2,ID00419637202311204720264_-10,2737.572510,915.239990
3,ID00419637202311204720264_-9,2741.034668,916.399414
4,ID00419637202311204720264_-8,2744.497559,917.558350
